In [5]:
import os,sys,glob
import pandas as pd
import numpy as np
import xarray as xr
import scipy
from scipy import stats
from scipy.special import erf
import matplotlib.pyplot as plt
import rioxarray

%load_ext autoreload
%autoreload 2

/home/ubuntu/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [6]:
from urb_object import *

In [7]:
city = 'Lissabon'
os.system('mkdir ../../validation_plots/%s' %(city))
os.system('mkdir ../../logs/%s' %(city))
os.system('mkdir ../../data_avoid_impacts_urbclim/%s' %(city))

256

In [8]:
indicators = []
for fl in sorted(glob.glob(data_path + 'Iconic_cities/Lissabon/*/*/*.csv')):
    if 'topography' not in fl:
        fl = fl.split('/')[-1].split('.')[0]
        indicators.append('_'.join(fl.split('_')[:-1]))
indicators = sorted(set(indicators))

In [1]:
indicators = [
    #'HWMI',
    #'LWH_int',
    #'LWH_light',
    #'LWH_mod',
    #'MTCM',
    #'MTWM',
    'T2M_daily_mean_max',
    'T2M_daily_mean_min',
    'T2M_dayover25',
    'T2M_dayover30',
    'T2M_dayover35',
    'T2M_mean',
    'T2M_nightover20',
    'T2M_nightover25',
    'T2M_nightover28',
    'WBGT_daily_mean_max',
    'WBGT_dayover25',
    'WBGT_dayover28',
    'WBGT_dayover295',
    'WBGT_dayover31',
    'WBGT_hourover25',
    'WBGT_hourover28',
    'WBGT_hourover295',
    'WBGT_hourover31',
    'WBGT_nightover25',
    'WBGT_nightover28',
    #'cooling_degree_hours',
    'heatwave_days',
    'heatwave_days_lissabon'
]

In [11]:
scenarios = ['CurPol','GS','SP']

In [12]:
study_locations = list(pd.read_table(data_path + 'daily_time_series/Lissabon/WBGT_timeseries_CurPol_enspctl95_2100.csv', sep=',').columns[1:])
dummy = rioxarray.open_rasterio(data_path + 'Iconic_cities/%s/Geotiffs/%s/%s_%s_%s_%s_EPSG4326.tif' \
                                                   %(city, 'CurPol', 'HWMI', '2021_2030', 'CurPol', 'ensmean'))
study_location_dict = {}
for l in study_locations:
    t,q,lat,lon = l.split('_')
    study_location_dict['%s_%s' %(t,q)] = {
        'y' : np.argmin(np.abs(dummy.y.values - float(lat))), 
        'x' : np.argmin(np.abs(dummy.x.values - float(lon)))}

study_locations = list(study_location_dict.keys()) + ['city_average']

In [ ]:
for indicator in indicators:
    out_file = '../../data_avoid_impacts_urbclim/%s/%s_%s.nc' %(city,city,indicator)
    if os.path.isfile(out_file):
        nc = xr.load_dataset(out_file)
        out = nc['data']
        approximation = nc['approximation']
        nc.close()
    else:
        out = xr.DataArray(dims=['study_location','scenario','quantile', 'year'], 
                           coords=dict(study_location=study_locations, 
                                       scenario=scenarios,
                                       quantile=[50,66,90],
                                       year=np.arange(2025,2100,10,'int')))
        approximation = xr.DataArray(dims=['study_location','scenario'], 
                               coords=dict(study_location=study_locations, 
                                       scenario=scenarios))
        approximation_dict = {'norm':0, 'skewnorm':1, 'fail':np.nan}
        approximation.attrs['explanation'] = str(approximation_dict)

    for study_location in study_locations:
        for scenario in scenarios:
            print(study_location,scenario)
            if np.any(np.isnan(out.loc[study_location, scenario])):
                urb = urb_object(
                    city=city, 
                    study_location=study_location,
                    indicator=indicator,
                    scenario=scenario)
                urb.load_data(coords=study_location_dict[study_location])
                urb.estimate_norm()
                urb.estimate_skewnorm()
                urb.plot()
                urb.evaluate_results_and_store_percentiles()
                urb.print_log()
                approximation.loc[study_location, scenario] = approximation_dict[urb._approximation]
                out.loc[study_location, scenario] = urb._out.values

    xr.Dataset({'data':out, 'approximation':approximation}).to_netcdf(out_file)

In [2]:
print(' '.join(indicators))

T2M_daily_mean_max T2M_daily_mean_min T2M_dayover25 T2M_dayover30 T2M_dayover35 T2M_mean T2M_nightover20 T2M_nightover25 T2M_nightover28 WBGT_daily_mean_max WBGT_dayover25 WBGT_dayover28 WBGT_dayover295 WBGT_dayover31 WBGT_hourover25 WBGT_hourover28 WBGT_hourover295 WBGT_hourover31 WBGT_nightover25 WBGT_nightover28 heatwave_days heatwave_days_lissabon


In [9]:
for indicator in indicators:
    print('nohup python prep_dashboard.py --city Lissabon --indicator %s > ../python_logs/%s_%s.log 2>&1 &' %(indicator,city,indicator))

nohup python prep_dashboard.py --city Lissabon --indicator HWMI > ../python_logs/Lissabon_HWMI.log 2>&1 &
nohup python prep_dashboard.py --city Lissabon --indicator LWH_int > ../python_logs/Lissabon_LWH_int.log 2>&1 &
nohup python prep_dashboard.py --city Lissabon --indicator LWH_light > ../python_logs/Lissabon_LWH_light.log 2>&1 &
nohup python prep_dashboard.py --city Lissabon --indicator LWH_mod > ../python_logs/Lissabon_LWH_mod.log 2>&1 &
nohup python prep_dashboard.py --city Lissabon --indicator MTCM > ../python_logs/Lissabon_MTCM.log 2>&1 &
nohup python prep_dashboard.py --city Lissabon --indicator MTWM > ../python_logs/Lissabon_MTWM.log 2>&1 &
nohup python prep_dashboard.py --city Lissabon --indicator T2M_daily_mean_max > ../python_logs/Lissabon_T2M_daily_mean_max.log 2>&1 &
nohup python prep_dashboard.py --city Lissabon --indicator T2M_daily_mean_min > ../python_logs/Lissabon_T2M_daily_mean_min.log 2>&1 &
nohup python prep_dashboard.py --city Lissabon --indicator T2M_dayover25 >